In [ ]:
from __future__ import division
import os, sys
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
import datetime
import pickle
from lightgbm import *
from sklearn.model_selection import *
from sklearn.ensemble import *
from sklearn.base import clone
import scipy
from sklearn.metrics import *
from catboost import *
from sklearn.preprocessing import *
from sklearn.pipeline import *
from sklearn.linear_model import *
from xgboost import *
from sklearn.tree import *
import pickle

In [ ]:
dateparse = lambda x: pd.datetime.strptime(x, '%d/%m/%y')
trainDFOriginal = pd.read_csv('dataset/train.csv', parse_dates=['booking_date', 'checkin_date', 'checkout_date'], date_parser=dateparse)
print('Done loading training data')
testDFOriginal = pd.read_csv('dataset/test.csv', parse_dates=['booking_date', 'checkin_date', 'checkout_date'], date_parser=dateparse)
print('Done loading testing data')

In [ ]:
X = X4
y = y4
XTest = XTest4
testIds = testIds4

In [ ]:
gfCV5 = list(GroupKFold(n_splits=5).split(X4, y4, groups4))
gfCV10 = list(GroupKFold(n_splits=10).split(X4, y4, groups4))

In [ ]:
def convertCat(trainDF, testDF, col):
    trainX = np.array(trainDF[col].values)
    testX = np.array(testDF[col].values)
    totX = np.vstack((trainX.reshape((-1, 1)), testX.reshape((-1, 1))))
    
    totSeries = pd.Series(totX.reshape((-1,)))
    categories = totSeries.astype('category').cat.categories
    
    print(categories)

    trainDF[col] = trainDF[col].astype('category').cat.set_categories(categories)
    testDF[col] = testDF[col].astype('category').cat.set_categories(categories)


In [ ]:
catFeatures = [
 'channel_code',
 'main_product_code',
 'persontravellingid',
 'resort_region_code',
 'resort_type_code',
 'room_type_booked_code',
 'season_holidayed_code',
 'state_code_residence',
 'state_code_resort',
 'member_age_buckets',
 'booking_type_code',
 'cluster_code',
 'reservationstatusid_code',
 'resort_id']


for feature in catFeatures:
    convertCat(trainDF, testDF, feature)

In [ ]:
# season_holidayed_code            341310 non-null category
# state_code_residence             336660 non-null category

trainDF['season_holidayed_code'].fillna(2.0, inplace=True)
trainDF['state_code_residence'].fillna(8.0, inplace=True)

testDF['season_holidayed_code'].fillna(2.0, inplace=True)
testDF['state_code_residence'].fillna(8.0, inplace=True)

# trainDF['season_holidayed_code'].value_counts()
# trainDF['state_code_residence'].value_counts()

In [ ]:
dir(trainDF['checkin_date'].dt)
newFeatures = ['day', 'dayofweek', 'dayofyear', 'days_in_month', 'is_leap_year', 'is_month_end',
 'is_month_start', 'is_quarter_end', 'is_quarter_start', 'is_year_end','is_year_start','month',
 'quarter','weekofyear', 'year']



In [ ]:
########################checkin, checkout and booking dates##########################################

for feature in newFeatures:
    trainDF['xxx_checkin_'+feature] = getattr(trainDF['checkin_date'].dt, feature)
for feature in newFeatures:
    trainDF['xxx_checkout_'+feature] = getattr(trainDF['checkout_date'].dt, feature)
for feature in newFeatures:
    trainDF['xxx_booking_'+feature] = getattr(trainDF['booking_date'].dt, feature)
    
for feature in newFeatures:
    testDF['xxx_checkin_'+feature] = getattr(testDF['checkin_date'].dt, feature)
for feature in newFeatures:
    testDF['xxx_checkout_'+feature] = getattr(testDF['checkout_date'].dt, feature)
for feature in newFeatures:
    testDF['xxx_booking_'+feature] = getattr(testDF['booking_date'].dt, feature)
    

    

In [ ]:
########################checkin, checkout and booking dates##########################################

trainDF['xxx_checkin_boarding_diff'] = (trainDF['checkin_date']-trainDF['booking_date']).dt.days
testDF['xxx_checkin_boarding_diff'] = (testDF['checkin_date']-testDF['booking_date']).dt.days

trainDF['xxx_checkout_checkin_diff'] = (trainDF['checkout_date']-trainDF['checkin_date']).dt.days
testDF['xxx_checkout_checkin_diff'] = (testDF['checkout_date']-testDF['checkin_date']).dt.days


trainDF['xxx_new_checkin'] = trainDF['checkin_date'].dt.date-datetime.date(2012, 3, 21)
testDF['xxx_new_checkin'] = testDF['checkin_date'].dt.date-datetime.date(2012, 3, 21)

trainDF['xxx_new_checkin'] = trainDF['xxx_new_checkin'].dt.days
testDF['xxx_new_checkin'] = testDF['xxx_new_checkin'].dt.days

trainDF['xxx_stayed_minus_roomnights'] = trainDF['roomnights']-trainDF['xxx_checkout_checkin_diff']
testDF['xxx_stayed_minus_roomnights'] = testDF['roomnights']-testDF['xxx_checkout_checkin_diff']




In [ ]:
del trainDF['checkin_date']
del testDF['checkin_date']
del trainDF['checkout_date']
del testDF['checkout_date']
del trainDF['booking_date']
del testDF['booking_date']

In [ ]:
y = np.array(trainDF['amount_spent_per_room_night_scaled'].values)
testIds = np.array(testDF['reservation_id'].values)
groups = np.array(trainDF['memberid'].values)

In [ ]:
##################################numberofadults, numberofchildren#####################################

trainDF['xxx_diff_people'] = -trainDF['numberofchildren']-trainDF['numberofadults']+trainDF['total_pax'] 
testDF['xxx_diff_people'] = -testDF['numberofchildren']-testDF['numberofadults']+testDF['total_pax']

trainDF['xxx_child_plus_adults'] = trainDF['numberofchildren'] + trainDF['numberofadults']
testDF['xxx_child_plus_adults'] = testDF['numberofchildren'] + testDF['numberofadults']

trainDF['xxx_child_fraction'] = trainDF['numberofchildren']/trainDF['xxx_child_plus_adults']
testDF['xxx_child_fraction'] = testDF['numberofchildren']/testDF['xxx_child_plus_adults']

trainDF['xxx_adult_fraction'] = trainDF['numberofadults']/trainDF['xxx_child_plus_adults']
testDF['xxx_adult_fraction'] = testDF['numberofadults']/testDF['xxx_child_plus_adults']

trainDF['xxx_ignore_pax_or_ch_ad'] = (trainDF['xxx_child_plus_adults']==trainDF['total_pax'])
testDF['xxx_ignore_pax_or_ch_ad'] = (testDF['xxx_child_plus_adults']==testDF['total_pax'])

trainDF['xxx_adult_fraction'].fillna(0, inplace=True)
testDF['xxx_adult_fraction'].fillna(0, inplace=True)
trainDF['xxx_child_fraction'].fillna(0, inplace=True)
testDF['xxx_child_fraction'].fillna(0, inplace=True)

In [ ]:
###########################################memberid###################################################

memberTrain = trainDF['memberid'].value_counts()
memberTest = testDF['memberid'].value_counts()

memberFeatureTrain = [0]*len(trainDF)
memberFeatureTest = [0]*len(testDF)

for i in range(len(trainDF)):
    memberFeatureTrain[i] = memberTrain[trainDF.loc[i, 'memberid']]

for i in range(len(testDF)):
    memberFeatureTest[i] = memberTest[testDF.loc[i, 'memberid']]
  
trainDF['xxx_member_feature'] = memberFeatureTrain
testDF['xxx_member_feature'] = memberFeatureTest


In [ ]:
del trainDF['reservation_id']
del trainDF['amount_spent_per_room_night_scaled']
del testDF['reservation_id']
del trainDF['memberid']
del testDF['memberid']

In [ ]:
pickle.dump([trainDF, testDF, y, testIds, groups], open('freshPickles/catFeaturesdtypecat.pk', 'wb'))